In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
from pathlib import Path
from polsarpro.util import read_PSP_bin, read_T3
from polsarpro.decompositions import freeman, freeman_dask
from pathlib import Path
import os
import matplotlib.pyplot as plt
import numpy as np

# access c-psp bin files
os.environ["PATH"]+=os.pathsep+"/home/c_psp/Soft/bin/data_process_sngl/"
os.environ["PATH"]+=os.pathsep+"/home/c_psp/Soft/bin/data_convert/"
input_test_dir = Path("/data/psp/input/freeman_decomposition/")

# output_test_dir = Path("/data/psp/output/freeman_decomposition/out")
output_test_dir = Path("/data/res/freeman_cpsp")


In [ ]:
from polsarpro.devtools import parse_psp_parameter_string
import os

input_str = f"""id: {input_test_dir}
od: /data/res/freeman_cpsp
iodf: T3
nwr: 7
nwc: 7
ofr: 0
ofc: 0
fnr: 18432
fnc: 1248
errf: /tmp"""

result = parse_psp_parameter_string(input_str)
# os.system(f"freeman_decomposition.exe {result}")
# 44s

In [ ]:
T3 = read_T3(input_test_dir)

file_odd = output_test_dir / "Freeman_Odd.bin"
ps_c = read_PSP_bin(file_odd)
file_dbl = output_test_dir / "Freeman_Dbl.bin"
pd_c = read_PSP_bin(file_dbl)
file_vol = output_test_dir / "Freeman_Vol.bin"
pv_c = read_PSP_bin(file_vol)

In [ ]:
ps_da, pd_da, pv_da = freeman_dask(T3, "T3", boxcar_size=[7, 7])

# Function T3_to_C3_dask took 12.1591 seconds
# Function boxcar_dask took 13.3408 seconds
# Function span took 0.5975 seconds
# Function _compute_freeman_components took 6.7740 seconds
# Function freeman_dask took 32.2872 seconds

In [ ]:
ps, pd, pv = freeman(T3, "T3", boxcar_size=[7, 7])

# Function T3_to_C3 took 9.7849 seconds
# Function boxcar took 47.6211 seconds
# Function span took 1.1133 seconds
# Function _compute_freeman_components took 8.1443 seconds
# Function freeman took 65.5946 seconds

In [ ]:
print(f"difference between dask and normal implementation for odd {np.abs(ps - ps_da).sum()}, double {np.abs(pd - pd_da).sum()}, and volume {np.abs(pv - pv_da).sum()}")

In [ ]:
var_name = "P_v"
var_p = pv_da
var_c = pv_c

plt.figure(figsize=(10, 10))
plt.imshow(var_p[::8][1000:1500], interpolation="none", vmax=1)
plt.colorbar(fraction=0.046, pad=0.04)
plt.title(f"${var_name}$ python")

plt.figure(figsize=(10, 10))
plt.imshow(var_c[::8][1000:1500], interpolation="none", vmax=1)
plt.colorbar(fraction=0.046, pad=0.04)
plt.title(f"${var_name}$ C")

np.nanmean(((np.abs(var_p - var_c) / np.abs(var_c))))

In [ ]:
var_name = "P_s"
var_p = ps_da
var_c = ps_c

plt.figure(figsize=(10, 10))
plt.imshow(var_p[::8][1000:1500], interpolation="none", vmax=1)
plt.colorbar(fraction=0.046, pad=0.04)
plt.title(f"${var_name}$ python")

plt.figure(figsize=(10, 10))
plt.imshow(var_c[::8][1000:1500], interpolation="none", vmax=1)
plt.colorbar(fraction=0.046, pad=0.04)
plt.title(f"${var_name}$ C")

np.nanmean(((np.abs(var_p - var_c) / np.abs(var_c))))

In [ ]:
var_name = "P_d"
var_p = pd_da
var_c = pd_c

plt.figure(figsize=(10, 10))
plt.imshow(var_p[::8][1000:1500], interpolation="none", vmin=0, vmax=1)
plt.colorbar(fraction=0.046, pad=0.04)
plt.title(f"${var_name}$ python")

plt.figure(figsize=(10, 10))
plt.imshow(var_c[::8][1000:1500], interpolation="none", vmax=1)
plt.colorbar(fraction=0.046, pad=0.04)
plt.title(f"${var_name}$ C")

np.nanmean(((np.abs(var_p - var_c) / np.abs(var_c))))